In [29]:
import io
import os
import shutil
import sys
import time
from collections import defaultdict

sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../../ajna_docs/commons')
sys.path.insert(0, '../../../../virasana')

from bhadrasana.main import mongodb as db
from ajna_commons.utils.images import mongo_image
from PIL import Image
from datetime import date, datetime, timedelta



In [47]:
#if os.path.exists('yolo'):
#    shutil.rmtree('yolo')


In [45]:
start = datetime(2020, 3, 2)
end = datetime.today()
limit = 1000
query = {'metadata.contentType': 'image/jpeg',
         'metadata.predictions.bbox': {'$exists': True},
         'metadata.dataescaneamento': {'$gte': start, '$lt': end}}
projection = {'_id': 1, 'metadata.predictions.bbox': 1}
# r = requests.post('https://ajna.labin.rf08.srf/virasana/grid_data', json=params, verify=False)


In [46]:
MIN_RATIO = 1.8
if not os.path.exists('yolo'):
    os.mkdir('yolo')
    
cursor = db.fs.files.find(query, projection).limit(limit)
for count, row in enumerate(cursor):
    _id = row['_id']
    predictions = row['metadata']['predictions']
    caminho_atual = os.path.join('yolo', str(_id))
    if os.path.exists(caminho_atual):
        print(str())
        continue
    image = Image.open(io.BytesIO(mongo_image(db, _id)))
    xfinal, yfinal = image.size
    if xfinal / yfinal < MIN_RATIO:
        print(image.size, ' - abortando...')
        continue
    os.mkdir(caminho_atual)
    arquivo_atual = os.path.join(caminho_atual, str(_id))
    with open(arquivo_atual + '.txt', 'w') as out_handle:
        for prediction in predictions:
            bbox = prediction['bbox']
            print(bbox, image.size)
            coords = []
            coords.append(str((bbox[1] + bbox[3]) / 2 / xfinal))
            coords.append(str((bbox[0] + bbox[2]) / 2 / yfinal))
            coords.append(str((bbox[3] - bbox[1]) / xfinal))
            coords.append(str((bbox[2] - bbox[0]) / yfinal))
            out_handle.write('0 ' + ' '.join(coords) + '\n')
    with open(os.path.join(caminho_atual, 'classes.txt'), 'w') as out_handle:
        out_handle.write('0 Container')
    image.save(arquivo_atual + '.jpg')
print('%s arquivos exportados...')

[52, 69, 657, 1325] (1684, 799)
[61, 38, 635, 1498] (1892, 799)
[37, 180, 626, 1564] (2155, 799)
[49, 61, 642, 1140] (1677, 799)
[51, 83, 662, 1432] (1986, 799)
[44, 127, 648, 1471] (1986, 799)
[79, 1111, 655, 2007] (2290, 799)
[63, 139, 646, 1024] (2290, 799)
(1381, 879)  - abortando...
[45, 69, 631, 1118] (1681, 799)
[17, 48, 640, 1672] (1767, 799)
[32, 181, 634, 1446] (2051, 799)
[56, 194, 650, 2009] (2309, 799)
[33, 94, 648, 1902] (2314, 799)
[54, 232, 638, 1889] (2128, 799)
[45, 253, 656, 1779] (2128, 799)
[89, 41, 642, 720] (1757, 799)
[75, 755, 655, 1442] (1757, 799)
[56, 32, 685, 3379] (3522, 799)
(1333, 799)  - abortando...


KeyboardInterrupt: 